In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm import tqdm

import kalepy as kale

import holodeck as holo
import holodeck.librarian
from holodeck.constants import *
from holodeck import cosmo, plot, utils, log

In [ ]:
path = (
    "/Users/lzkelley/Programs/nanograv/holodeck/output/"
    "doppler_astro-strong-all_n1080_r100_f40"
)
path = Path(path)
assert path.is_dir()

In [ ]:
space, space_fname = holo.librarian.lib_tools.load_pspace_from_path(path)
args, args_fname = holo.librarian.gen_lib.load_config_from_path(path, log)
def_params = space.default_params()
sam, hard = space.model_for_params(def_params, sam_shape=args.sam_shape)

In [ ]:
pattern = "library_sims/*.npz"
files = list(path.glob(pattern))
num_files = len(files)
print(f"Found {num_files} matches to '{pattern}' in '{path}'")

num_det_mqf = None
num_all_mqf = None
num_det_z = None
num_all_z = None
doppler_fobs_gw_cents = None
fobs_gw_cents = None
num_has_doppler = 0
fits_det = []
fits_undet = []
for ii, fil in enumerate(tqdm(files)):
    data = np.load(fil, allow_pickle=True)
    keys = data.keys()
    if ii == 0:
        print(list(keys))

    if 'fail' in keys:
        # print(f"{ii=:04d} = failure | '{fil}'")
        continue
    fit = data['psd_fit']
    # print(fit)
    has_doppler = 'doppler_detect' in keys
    if not has_doppler:
        fits_undet.append(fit)
        continue

    fits_det.append(fit)
    doppler_fobs_gw_cents = data['doppler_fobs_gw_cents']
    fobs_gw_cents = data['fobs_cents']

    if num_det_mqf is None:
        mqf_shape = data['doppler_num_det'].shape
        num_det_mqf = np.zeros((num_files,) + mqf_shape)
        num_all_mqf = np.zeros_like(num_det_mqf)
        z_shape = data['doppler_num_det_redz'].shape
        num_det_z = np.zeros((num_files,) + z_shape)
        num_all_z = np.zeros_like(num_det_z)
        gwb_shape = data['gwb'].shape
        gwb_det = np.zeros((num_files,) + gwb_shape)

    num_det_mqf[num_has_doppler, ...] = data['doppler_num_det'][()]
    num_all_mqf[num_has_doppler, ...] = data['doppler_num_all'][()]
    num_det_z[num_has_doppler, ...] = data['doppler_num_det_redz'][()]
    num_all_z[num_has_doppler, ...] = data['doppler_num_all_redz'][()]
    gwb_det[num_has_doppler, ...] = data['gwb'][()]
    num_has_doppler += 1

num_det_mqf = num_det_mqf[:num_has_doppler, ...]
num_all_mqf = num_all_mqf[:num_has_doppler, ...]
num_det_z = num_det_z[:num_has_doppler, ...]
num_all_z = num_all_z[:num_has_doppler, ...]
gwb_det = gwb_det[:num_has_doppler, ...]
fits_undet = np.asarray(fits_undet)
fits_det = np.asarray(fits_det)


In [ ]:
doppler_fname = path.name + "_detected.npz"
doppler_fname = path.joinpath(doppler_fname)

"""
Values:
    # ---- Semi-Analytic Model
    mtot : (M+1,) total-mass bin edges [gram]
    mrat : (Q+1,) mass-ratio bin edges
    redz : (Z+1,) redshift (of galaxy merger)

    # ---- PTA calculations
    fobs_gw_cents : (Fp,) GW frequency-bin centers
    gwb : (S, Fp, R), GWB characteristic strain in PTA band
    fits : (S, 2) power-law fits to first 5 frequency bins of PTA GWB, 0=amplitude, 1=spectral-index

    # ---- Doppler calculations
    doppler_fobs_gw_cents : (Fd,) Doppler frequency-bin centers
    num_det_mqf : (S, M, Q, Fd) number of detected binaries in bins of mtot, mrat, freq (doppler GW frequency)
    num_det_z : (S, Z) number of detected binaries in bins of redz (of galaxy merger)

Array shapes have the sizes:
    M : number of total-mass bins = 90
    Q : number of mass-ratio bins = 80
    Z : number of redshift bins = 100
    S : populations consistent with GWB observations = 140 (out of 1080 explored)
    Fp : PTA frequencies = 40
    R : number of realizations per population = 100
    Fd : Doppler frequencies = 200

"""

np.savez(
    doppler_fname,

    # (M+1,) total-mass bin edges [gram]
    mtot=sam.mtot,
    # (Q+1,) mass-ratio bin edges
    mrat=sam.mrat,
    # (Z+1,) redshift (of galaxy merger)
    redz=sam.redz,

    # ---- PTA calculations
    # (Fp,) GW frequency-bin centers
    fobs_gw_cents=fobs_gw_cents,
    # (S, Fp, R), GWB characteristic strain in PTA band
    gwb=gwb_det,
    # (S, 2) power-law fits to first 5 frequency bins of PTA GWB, 0=amplitude, 1=spectral-index
    fits=fits_det,

    # ---- Doppler calculations
    # (Fd,) Doppler frequency-bin centers
    doppler_fobs_gw_cents=doppler_fobs_gw_cents,
    # (S, M, Q, Fd) number of detected binaries in bins of mtot, mrat, freq (doppler GW frequency)
    num_det_mqf=num_det_mqf,
    # (S, Z) number of detected binaries in bins of redz (of galaxy merger)
    num_det_z=num_det_z,

)
print(f"Saved to {doppler_fname}, {utils.get_file_size(doppler_fname)}")

In [ ]:
num_det_mqf.shape, num_det_z.shape

In [ ]:
fig, ax = plot.figax(scale='lin', xlabel=r'$\log_{10}A_\mathrm{yr}$', ylabel=r'$\gamma_\mathrm{PSD}$')

ax.scatter(*fits_undet.T, color='grey', alpha=0.2, s=10)
ax.scatter(*fits_det.T, color='C1', alpha=0.4, s=20)

ax.axhline(-13/3.0, color='k', ls='--', alpha=0.5)
ax.axvline(np.log10(2.5e-15), color='k', ls='--', alpha=0.5)

rect = mpl.patches.Rectangle((-15, -5), 1.0, 2.5, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)

plt.show()

In [ ]:
utils.gamma_psd_to_strain(-5.0), utils.gamma_psd_to_strain(-2.5)

In [ ]:
fig, ax = plot.figax(scale='lin', xlabel=r'$\log_{10}(\mathrm{number} \;\; \mathrm{detected})$', ylabel='density')

xedges = np.linspace(0, 4, 41)

xx = num_det_mqf.sum(axis=(1, 2, 3))
med = np.median(xx)

kale.dist1d(np.log10(xx), edges=xedges, ax=ax, density=True, carpet=False)

ax.axvline(np.log10(med), color='C1', ls='-', lw=1.0, alpha=0.75)
conf = np.log10(np.percentile(xx, [25, 75]))
print(f"{10.0**conf[0]:.0f}, {med:.0f}, {10.0**conf[1]:.0f}")
for cc in conf:
    ax.axvline(cc, color='C1', ls='--', lw=0.5, alpha=0.5)

ax.axvspan(*conf, color='C1', alpha=0.1)

fig.savefig('2.png')
plt.show()


In [ ]:
fig, ax = plot.figax(
    scale='log',
    xlabel=r'Binary Total Mass $[M_\odot]$', xlim=[1e6, 1e11],
    ylabel=r'Number', ylim=[1e-4, 1e2],
)

yy = num_det_mqf.sum(axis=(2, 3))
dx = np.diff(np.log10(sam.mtot))
xx = utils.midpoints(sam.mtot/MSOL, log=True)

med = np.median(yy, axis=0)/dx
ax.plot(xx, med, label=r"$dN \, / \, d \, \log_{10}M$")
conf = np.percentile(yy, [25, 75], axis=0)/dx
ax.fill_between(xx, *conf, alpha=0.2)

yy = np.cumsum(yy, axis=-1)
med = np.median(yy, axis=0)
ax.plot(xx, med, label=r'$N(\less M)$')
conf = np.percentile(yy, [25, 75], axis=0)
ax.fill_between(xx, *conf, alpha=0.2)

ax.legend()
plt.show()


In [ ]:
fig, ax = plot.figax(
    scale='log',
    xlabel=r'Binary Mass Ratio',
    ylabel=r'Number',
)

yy = num_det_mqf.sum(axis=(1, 3))
dx = np.diff(np.log10(sam.mrat))
xx = utils.midpoints(sam.mrat, log=True)

med = np.median(yy, axis=0)/dx
ax.plot(xx, med, label=r"$dN \, / \, d \, \log_{10}q$")
conf = np.percentile(yy, [25, 75], axis=0)/dx
ax.fill_between(xx, *conf, alpha=0.2)

yy = np.cumsum(yy, axis=-1)
med = np.median(yy, axis=0)
ax.plot(xx, med, label=r'$N(\less q)$')
conf = np.percentile(yy, [25, 75], axis=0)
ax.fill_between(xx, *conf, alpha=0.2)

ax.legend()
fig.savefig('3b.png')
plt.show()


In [ ]:
fig, ax = plot.figax(
    scale='log',
    xlabel=r'GW Frequency $[\mathrm{nHz}]$',
    ylabel=r'Number',
)

yy = num_det_mqf.sum(axis=(1, 2))
# xx = utils.midpoints(sam.mrat, log=True)
xx = doppler_fobs_gw_cents * 1e9
# dx = np.diff(np.log10(sam.mrat))
dx = xx[0]

med = np.median(yy, axis=0)/dx
ax.plot(xx, med, label=r"$dN \, / \, d\, \ln f$")
conf = np.percentile(yy, [25, 75], axis=0)/dx
ax.fill_between(xx, *conf, alpha=0.2)

yy = np.cumsum(yy, axis=-1)
med = np.median(yy, axis=0)
ax.plot(xx, med, label=r'$N(\less f)$')
conf = np.percentile(yy, [25, 75], axis=0)
ax.fill_between(xx, *conf, alpha=0.2)

ax.legend()
fig.savefig('5.png')
plt.show()


In [ ]:
doppler_path = Path("/Users/lzkelley/Programs/nanograv/holodeck/output/doppler")
nums = []
for file in doppler_path.glob("*.npz"):
    with np.load(file, allow_pickle=True) as data:
        det = data['num_det_mqf']
        nums.append(det.shape[0])
